### Better Prediction with CatBoost Regressor

In [17]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from datetime import datetime
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge,Lasso,RidgeCV,LassoCV,ElasticNet,ElasticNetCV,LinearRegression
from sklearn.model_selection import train_test_split,GridSearchCV
sns.set()
import pylab
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from IPython.display import Image
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import f1_score as f1
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,GradientBoostingRegressor
from catboost import CatBoostClassifier,CatBoostRegressor
from lightgbm import LGBMClassifier,LGBMRegressor
from xgboost import XGBClassifier,XGBRegressor
from sklearn.ensemble import AdaBoostRegressor

In [2]:
df=pd.read_csv("C:/Users/yashw/Downloads/Admission_Prediction.csv")

In [3]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332.0,108.0,5.0,4.5,4.0,9.02,1,0.87
496,497,337.0,117.0,5.0,5.0,5.0,9.87,1,0.96
497,498,330.0,120.0,5.0,4.5,5.0,9.56,1,0.93
498,499,312.0,103.0,4.0,4.0,5.0,8.43,0,0.73


In [11]:
df.isnull().sum()

GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [5]:
df['GRE Score']=df['GRE Score'].fillna(df['GRE Score'].median())

In [6]:
df['TOEFL Score']=df['TOEFL Score'].fillna(df['TOEFL Score'].median())

In [7]:
df['University Rating']=df['University Rating'].fillna(df['University Rating'].median())

In [8]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,317.0,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332.0,108.0,5.0,4.5,4.0,9.02,1,0.87
496,497,337.0,117.0,5.0,5.0,5.0,9.87,1,0.96
497,498,330.0,120.0,5.0,4.5,5.0,9.56,1,0.93
498,499,312.0,103.0,4.0,4.0,5.0,8.43,0,0.73


In [9]:
df=df.drop(columns='Serial No.')

In [10]:
df

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,317.0,104.0,3.0,3.0,3.5,8.00,1,0.72
3,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...
495,332.0,108.0,5.0,4.5,4.0,9.02,1,0.87
496,337.0,117.0,5.0,5.0,5.0,9.87,1,0.96
497,330.0,120.0,5.0,4.5,5.0,9.56,1,0.93
498,312.0,103.0,4.0,4.0,5.0,8.43,0,0.73


## Applying AdaBoost Regressor

In [12]:
X=df.drop(columns='Chance of Admit')
Y=df['Chance of Admit']

In [20]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.20,random_state=234)

In [21]:
lm=AdaBoostRegressor()
lm.fit(x_train,y_train)

AdaBoostRegressor()

In [22]:
y_pred=lm.predict(x_test)

In [23]:
y_pred

array([0.58737903, 0.66113924, 0.52971831, 0.61745665, 0.82      ,
       0.75831325, 0.73323529, 0.77847458, 0.63294118, 0.72226804,
       0.66128713, 0.58704545, 0.66113924, 0.60333333, 0.63288462,
       0.62242574, 0.53634783, 0.75785714, 0.62881356, 0.75785714,
       0.72245902, 0.56876543, 0.758     , 0.758     , 0.76380952,
       0.62454545, 0.54895735, 0.61120482, 0.78035714, 0.73232558,
       0.60050251, 0.90787879, 0.65666667, 0.58234043, 0.76542857,
       0.61602564, 0.55363095, 0.61745665, 0.56876543, 0.90787879,
       0.53634783, 0.51265625, 0.7652381 , 0.864     , 0.63294118,
       0.81      , 0.60932489, 0.77603175, 0.87368421, 0.51265625,
       0.89352941, 0.61120482, 0.62454545, 0.864     , 0.88891892,
       0.66037037, 0.47204545, 0.62068966, 0.90711111, 0.88153846,
       0.72571429, 0.86676471, 0.84152174, 0.51265625, 0.62068966,
       0.66128713, 0.61086957, 0.64670807, 0.61086957, 0.63481283,
       0.73232558, 0.75831325, 0.65575949, 0.66128713, 0.58737

In [24]:
lm.score(x_test,y_test)

0.6277956773392452

In [25]:
def adj_r2(x,y):
    r2=lm.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [26]:
adj_r2(x_test,y_test)

0.5994757832237531

In [27]:
print('MSE: ',mean_squared_error(y_test,y_pred))
print('MAPE: ',mean_absolute_percentage_error(y_test, y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

MSE:  0.005933424490875116
MAPE:  0.08943489988962053
MAE:  0.06460979118484377


## Applying GradientBoost Regressor

In [29]:
lm=GradientBoostingRegressor()
lm.fit(x_train,y_train)

GradientBoostingRegressor()

In [30]:
y_pred=lm.predict(x_test)

In [32]:
y_pred

array([0.61885831, 0.6936764 , 0.58799898, 0.62034496, 0.85945285,
       0.7651953 , 0.74981223, 0.8180873 , 0.67757422, 0.71387049,
       0.68318456, 0.5986114 , 0.70100971, 0.62046834, 0.70856631,
       0.64462095, 0.54365334, 0.74279013, 0.67563801, 0.76015206,
       0.71858542, 0.61885831, 0.76923026, 0.77342937, 0.78742356,
       0.61493476, 0.56950655, 0.6067855 , 0.85048028, 0.75737656,
       0.62448165, 0.9380072 , 0.65560235, 0.56521409, 0.78764535,
       0.64921406, 0.58237267, 0.65259945, 0.61720068, 0.9566265 ,
       0.52884006, 0.52551448, 0.80114199, 0.90053383, 0.63332645,
       0.84329388, 0.61721752, 0.82703069, 0.89796606, 0.53031232,
       0.93134642, 0.64016397, 0.62423765, 0.93214748, 0.93339289,
       0.6741084 , 0.47048652, 0.62793452, 0.94564963, 0.9192451 ,
       0.71083613, 0.90217512, 0.89052276, 0.55279765, 0.63005482,
       0.70887358, 0.6397451 , 0.67819139, 0.6348487 , 0.70386288,
       0.73649996, 0.77243158, 0.69184506, 0.68352529, 0.60295

In [33]:
lm.score(x_test,y_test)

0.7516661309313286

In [34]:
def adj_r2(x,y):
    r2=lm.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [35]:
adj_r2(x_test,y_test)

0.7327711626326254

In [36]:
print('MSE: ',mean_squared_error(y_test,y_pred))
print('MAPE: ',mean_absolute_percentage_error(y_test, y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

MSE:  0.003958767190323101
MAPE:  0.06693262174103606
MAE:  0.04638953436555418


## Applying XGBoost Regressor

In [39]:
lm=LGBMRegressor()
lm.fit(x_train,y_train)

LGBMRegressor()

In [40]:
y_pred=lm.predict(x_test)

In [41]:
y_pred

array([0.61381767, 0.65897957, 0.59219091, 0.60980688, 0.87009788,
       0.76195773, 0.73957573, 0.83349498, 0.65246254, 0.73131054,
       0.69839493, 0.56599171, 0.67873592, 0.62402862, 0.72179466,
       0.65285831, 0.51924924, 0.75463285, 0.68435908, 0.75713722,
       0.72304132, 0.611162  , 0.75115861, 0.79159836, 0.79403079,
       0.64896777, 0.5844108 , 0.63074046, 0.86196323, 0.78500028,
       0.62841531, 0.94913065, 0.69392523, 0.53407921, 0.79743407,
       0.64674429, 0.56340685, 0.6647749 , 0.59678846, 0.95659856,
       0.5072097 , 0.51122994, 0.79124277, 0.88998223, 0.63165646,
       0.85303951, 0.59287251, 0.83458917, 0.90583463, 0.53797009,
       0.93869947, 0.62816312, 0.62147632, 0.91564674, 0.94161612,
       0.75255536, 0.45954178, 0.67442049, 0.9516287 , 0.91320212,
       0.71509843, 0.89059338, 0.88360472, 0.54214352, 0.64459499,
       0.69154725, 0.61195926, 0.68927968, 0.60736576, 0.70936988,
       0.71116786, 0.79803305, 0.67280565, 0.6711709 , 0.60591

In [42]:
lm.score(x_test,y_test)

0.7309555301942025

In [43]:
def adj_r2(x,y):
    r2=lm.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [44]:
adj_r2(x_test,y_test)

0.7104847553176743

In [45]:
print('MSE: ',mean_squared_error(y_test,y_pred))
print('MAPE: ',mean_absolute_percentage_error(y_test, y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

MSE:  0.004288921296959859
MAPE:  0.07066037930241918
MAE:  0.04865566058967152


## Applying CatBoost Regressor

In [47]:
lm=CatBoostRegressor()
lm.fit(x_train,y_train)

Learning rate set to 0.035425
0:	learn: 0.1406339	total: 152ms	remaining: 2m 32s
1:	learn: 0.1373861	total: 155ms	remaining: 1m 17s
2:	learn: 0.1344514	total: 158ms	remaining: 52.6s
3:	learn: 0.1316623	total: 161ms	remaining: 40.1s
4:	learn: 0.1287982	total: 164ms	remaining: 32.5s
5:	learn: 0.1258273	total: 166ms	remaining: 27.5s
6:	learn: 0.1231256	total: 169ms	remaining: 23.9s
7:	learn: 0.1205304	total: 172ms	remaining: 21.3s
8:	learn: 0.1179363	total: 174ms	remaining: 19.2s
9:	learn: 0.1153907	total: 177ms	remaining: 17.6s
10:	learn: 0.1130229	total: 180ms	remaining: 16.1s
11:	learn: 0.1108040	total: 182ms	remaining: 15s
12:	learn: 0.1085878	total: 184ms	remaining: 14s
13:	learn: 0.1062580	total: 186ms	remaining: 13.1s
14:	learn: 0.1042129	total: 188ms	remaining: 12.4s
15:	learn: 0.1022590	total: 190ms	remaining: 11.7s
16:	learn: 0.1003790	total: 192ms	remaining: 11.1s
17:	learn: 0.0986080	total: 194ms	remaining: 10.6s
18:	learn: 0.0968597	total: 197ms	remaining: 10.2s
19:	learn: 0.

In [49]:
y_pred=lm.predict(x_test)

In [50]:
y_pred

array([0.62700882, 0.68863266, 0.57271725, 0.62653187, 0.87498845,
       0.76432524, 0.75701583, 0.84079187, 0.6874872 , 0.73264239,
       0.67265889, 0.58197128, 0.68790098, 0.66298479, 0.72270961,
       0.61842812, 0.54325289, 0.72748705, 0.68329279, 0.75753483,
       0.72565016, 0.61112539, 0.75453176, 0.77230502, 0.78516777,
       0.62664827, 0.55357903, 0.58671299, 0.84078563, 0.77952136,
       0.62465262, 0.9404423 , 0.62179575, 0.58111568, 0.81650969,
       0.66258529, 0.57856178, 0.67619656, 0.5842735 , 0.96474447,
       0.57721523, 0.53577832, 0.7873207 , 0.88063572, 0.65697214,
       0.8351754 , 0.55973943, 0.83770543, 0.91213272, 0.53822353,
       0.94293854, 0.65136297, 0.61226248, 0.92108079, 0.93499125,
       0.68291783, 0.45663247, 0.63073813, 0.9514052 , 0.9128424 ,
       0.70626394, 0.89654296, 0.88081251, 0.5356389 , 0.65210542,
       0.68985237, 0.59731405, 0.69451297, 0.6311467 , 0.70829131,
       0.73761605, 0.76980643, 0.69697239, 0.6946828 , 0.54201

In [51]:
lm.score(x_test,y_test)

0.766334922928102

In [52]:
def adj_r2(x,y):
    r2=lm.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [53]:
adj_r2(x_test,y_test)

0.7485560583682838

In [54]:
print('MSE: ',mean_squared_error(y_test,y_pred))
print('MAPE: ',mean_absolute_percentage_error(y_test, y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

MSE:  0.00372492742977702
MAPE:  0.06301792127118427
MAE:  0.044073741787353876


## Applying LightBoost Regressor

In [57]:
lm=LGBMRegressor()
lm.fit(x_train,y_train)

LGBMRegressor()

In [58]:
y_pred=lm.predict(x_test)

In [59]:
y_pred

array([0.61381767, 0.65897957, 0.59219091, 0.60980688, 0.87009788,
       0.76195773, 0.73957573, 0.83349498, 0.65246254, 0.73131054,
       0.69839493, 0.56599171, 0.67873592, 0.62402862, 0.72179466,
       0.65285831, 0.51924924, 0.75463285, 0.68435908, 0.75713722,
       0.72304132, 0.611162  , 0.75115861, 0.79159836, 0.79403079,
       0.64896777, 0.5844108 , 0.63074046, 0.86196323, 0.78500028,
       0.62841531, 0.94913065, 0.69392523, 0.53407921, 0.79743407,
       0.64674429, 0.56340685, 0.6647749 , 0.59678846, 0.95659856,
       0.5072097 , 0.51122994, 0.79124277, 0.88998223, 0.63165646,
       0.85303951, 0.59287251, 0.83458917, 0.90583463, 0.53797009,
       0.93869947, 0.62816312, 0.62147632, 0.91564674, 0.94161612,
       0.75255536, 0.45954178, 0.67442049, 0.9516287 , 0.91320212,
       0.71509843, 0.89059338, 0.88360472, 0.54214352, 0.64459499,
       0.69154725, 0.61195926, 0.68927968, 0.60736576, 0.70936988,
       0.71116786, 0.79803305, 0.67280565, 0.6711709 , 0.60591

In [60]:
lm.score(x_test,y_test)

0.7309555301942025

In [61]:
def adj_r2(x,y):
    r2=lm.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [62]:
adj_r2(x_test,y_test)

0.7104847553176743

In [63]:
print('MSE: ',mean_squared_error(y_test,y_pred))
print('MAPE: ',mean_absolute_percentage_error(y_test, y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

MSE:  0.004288921296959859
MAPE:  0.07066037930241918
MAE:  0.04865566058967152
